In [ ]:
# # verstack requirements.txt file:

# !pip install \
#     "numpy>=1.26.4,<=2.1.1" \
#     "pandas==2.2.2" \
#     "scikit-learn>=1.3.2,<=1.5.1" \
#     "lightgbm>=4.4.0,<=4.5.0" \
#     "optuna>=3.5.0,<=4.0.0" \
#     "optuna-integration>=3.2.0,<=4.0.0" \
#     "plotly>=5.11.0,<=5.24.0" \
#     "matplotlib==3.9.2" \
#     "seaborn==0.13.2" \
#     "python-dateutil==2.9.0" \
#     "holidays==0.56" \
#     "mlxtend==0.23.1" \
#     "category_encoders>=2.5.1,<=2.6.3" \
#     "verstack"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate, train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.linear_model import LogisticRegression # Baseline
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import joblib
from verstack import NaNImputer

### Load preprocessed data (ready for modeling)

In [ ]:
train_preprocessed = pd.read_csv('https://raw.githubusercontent.com/MohamedMostafa259/Customer-Churn-Prediction-and-Analysis/refs/heads/main/Milestone2_FeatureEng_AdvancedAnalysis/data/train_split_preprocessed.csv')
train_preprocessed.head()

In [ ]:
train_preprocessed.isna().sum()

In [ ]:
X_train_preprocessed = train_preprocessed.drop('churn_risk_score', axis=1)
y_train = train_preprocessed['churn_risk_score']

### Selecting promising models

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'SVC': SVC(),
    'XGBoost': XGBClassifier(),
    'CatBoost': CatBoostClassifier(verbose=0),
    'LightGBM': LGBMClassifier(verbose=0)
}


for name, model in models.items():
    print(f"CV on {name}")
    cv_results = cross_validate(
        model,
        X_train_preprocessed,
# target labels (y_train_split) starts from '1': [1, 2, 3, 4, 5], 
# but XGBoost expects them to start from 0, like [0, 1, 2, 3, 4].
        y_train - 1,
# don't forget to add one in the prediction time:
# y_pred = xgb_clf.predict(X_test_preprocessed) + 1
        cv=4,
        scoring='accuracy',
        return_train_score=True
    )
    print(f"Train Accuracy: {cv_results['train_score'].mean():.4f} (+/- {cv_results['train_score'].std():.4f})")
    print(f"Validation Accuracy: {cv_results['test_score'].mean():.4f} (+/- {cv_results['test_score'].std():.4f})")
    print('-'*30)


It's clear that XGBoost, CatBoost, LightGBM are the best models. So, we will take them to next step, which hyperparamater tuning.

### Hyperparameter Tuning

In [ ]:
pipe = Pipeline([
    ('model', LogisticRegression()) # placeholder will be replaced by the best model
])

param_grid = [
    {
        'model': [XGBClassifier(verbosity=0, random_state=42)],
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [6, 10, 13],
        'model__subsample': [0.5, 0.75, 1],
    },
    {
        # `bootstrap_type='Bernoulli'` is required for `subsample` parameter to work
        'model': [CatBoostClassifier(verbose=0, random_state=42, bootstrap_type='Bernoulli')],
        'model__iterations': [100, 200],  # CatBoost uses iterations instead of n_estimators
        'model__learning_rate': [0.01, 0.1],
        'model__depth': [6, 10, 13],  # CatBoost uses depth instead of max_depth
        'model__subsample': [0.5, 0.75, 1],
    },
    {
        'model': [LGBMClassifier(verbose=-1, random_state=42)],
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [6, 10, 13],
        'model__subsample': [0.5, 0.75, 1],
    }
]

grid = RandomizedSearchCV(pipe, param_distributions=param_grid, cv=4, scoring='accuracy', 
                    verbose=1, return_train_score=True, n_iter=60, n_jobs=-1, 
                    error_score='raise', random_state=42)

grid.fit(X_train_preprocessed, y_train-1)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
best_model = grid.best_estimator_
best_model

In [ ]:
cv_results_df = pd.DataFrame(grid.cv_results_).sort_values('mean_test_score', ascending=False)
print(cv_results_df.shape)
cols_to_show = [ "rank_test_score", "mean_test_score", "std_test_score",
                "mean_train_score", "std_train_score", "param_model", 
                "param_model__subsample", "param_model__n_estimators",
                "param_model__max_depth", "param_model__learning_rate"
]
cv_results_df[cols_to_show].head(10)

In [ ]:
cv_results_df[(cv_results_df['param_model__subsample']==0.750) & (cv_results_df['param_model__n_estimators']==100) &\
                (cv_results_df['param_model__max_depth']==6) & (cv_results_df['param_model__learning_rate']==0.01)]

The best model was `XGBClassifier` with these parameters:
- 'subsample': 0.75,
- 'n_estimators': 200,
- 'max_depth': 6,
- 'learning_rate': 0.01

Let's see what is the performance of the same model with 'n_estimators'=100. 

In [ ]:
xgb_clf = XGBClassifier(verbosity=0, random_state=42, n_estimators=100, 
                       learning_rate=0.01, max_depth=6, subsample=0.75)
xgb_cv_results = cross_validate(xgb_clf, X_train_preprocessed, y_train-1, 
                                cv=4, scoring='accuracy', return_train_score=True)

In [ ]:
print(f"XGB Train Accuracy: {xgb_cv_results['train_score'].mean():.4f} (+/- {xgb_cv_results['train_score'].std():.4f})")
print(f"XGB Validation Accuracy: {xgb_cv_results['test_score'].mean():.4f} (+/- {xgb_cv_results['test_score'].std():.4f})")

In [ ]:
print('\nBest model from cross validation:')
cv_results_df[['mean_train_score', 'mean_test_score']].iloc[0]

It seems that the best model XGBClassifier, resulted from the Randomized Cross Validation above, performs better (or equivalent) when having less number of estimators: n_estimators=100 !!

So, let's stick with this simpler version.

#### Merging the selected ml model with the previous pipelines & transformers into one pipeline


In [ ]:
# `DataCleaner` and `NaNImputerWrapper` are needed for `cleaning_pipeline`
from custom_transformers import DataCleaner, NaNImputerWrapper, FeatureEng

In [ ]:
train_split_cleaned = pd.read_csv('https://raw.githubusercontent.com/MohamedMostafa259/Customer-Churn-Prediction-and-Analysis/main/Milestone1_DataCollection_EDA_DataCleaning/data/train_split_cleaned.csv')[['membership_category', 'feedback', 'points_in_wallet', 'churn_risk_score']]
val_split_cleaned = pd.read_csv('https://raw.githubusercontent.com/MohamedMostafa259/Customer-Churn-Prediction-and-Analysis/main/Milestone1_DataCollection_EDA_DataCleaning/data/validation_split_cleaned.csv')[['membership_category', 'feedback', 'points_in_wallet', 'churn_risk_score']]

In [ ]:
X_train_split = train_split_cleaned.drop('churn_risk_score', axis=1)
y_train_split = train_split_cleaned['churn_risk_score']

In [ ]:
X_val_split = val_split_cleaned.drop('churn_risk_score', axis=1)
y_val_split = val_split_cleaned['churn_risk_score']

In [ ]:
# ../ means go up one dir
cleaning_pipeline = joblib.load("../../Milestone1_DataCollection_EDA_DataCleaning/pipelines/cleaning_pipeline.joblib")
cleaning_pipeline

In [ ]:
pipeline = Pipeline([
    ('cleaning_pipeline', cleaning_pipeline),  
    ('feature_engineering', FeatureEng()), 
	('model', xgb_clf)
])

pipeline.fit(X_train_split, y_train_split-1)

In [ ]:
y_train_pred = pipeline.predict(X_train_split) + 1

In [ ]:
print('train_accuracy:', accuracy_score(y_train_split, y_train_pred), '\n')
print('train_classification_report:\n', classification_report(y_train_split, y_train_pred), '\n')

In [ ]:
y_val_pred = pipeline.predict(X_val_split) + 1

In [ ]:
print('test_accuracy:', accuracy_score(y_val_split, y_val_pred), '\n')
print('test_classification_report:\n', classification_report(y_val_split, y_val_pred), '\n')

### Summary of the model results

* **Overall test accuracy**: **0.788** → The model correctly predicts the class **\~79% of the time** across the entire test set.

* **F1-score (macro avg) = F1-score (weighted avg) = 0.77** → The model performs well overall despite imbalanced data. This score is equal to the score of the 1st winner of the competition made on this dataset. [He built a model with **0.77** f1_macro](https://www.hackerearth.com/challenges/new/competitive/hackerearth-machine-learning-challenge-predict-customer-churn/#:~:text=Machine%20Learning%20practice-,Winners,-Adarsh%20Wase).

    - The [**evaluation metric**](https://www.hackerearth.com/problem/machine-learning/predict-the-churn-risk-rate-11-fb7a760d/#:~:text=score%20%3D%20100%20x%20metrics.f1_score(actual%2C%20predicted%2C%20average%3D%22macro%22)) for the HackerEarth Machine Learning Challenge: "How NOT to Lose a Customer in 10 Days" was based on the macro-averaged F1 score. Specifically, the final score was calculated as: **Final Score = 100 × F1_macro**

---

**One critical limitation of the model:**

It has low recall (0.43) on class 4, leading to a low f1-score (0.57) on that class as well. 

As a result, over half of actual class 4 samples are misclassified. That’s unacceptable for a high-risk churn group.

**Solution** may be:

* increasing the predicted probability of class 4 

* More examples 

* Better features focusing on separating class 4

<br>

**Another limitation** (less dangerous):

It has low recall (0.59) on class 2; however, we care more about addressing higher risk scores (recall of 3, 4, and 5 scores). But, we should take this into consideration as well.

## Error Analysis

In [ ]:
y_proba_train = xgb_clf.predict_proba(X_train_preprocessed)

In [ ]:
error_analysis_df = pd.DataFrame(np.c_[y_proba_train, np.array(y_train_split), np.array(y_train_pred)], 
                                 columns=['class1_prob', 'class2_prob', 'class3_prob', 'class4_prob', 
                                          'class5_prob', 'y_train', 'y_pred']).round(2)

### Erros Analysis for class 4 samples that are misclassified

In [ ]:
error_analysis_df.loc[error_analysis_df['y_train']==4, ['y_train', 'y_pred']].value_counts()

More than half of the class 4 samples are misclassified, mostly as 5.0, followed by 3.0. This explains the terrible recall (0.43) for class 4.

In [ ]:
error_analysis_df.loc[(error_analysis_df['y_train']==4) & (error_analysis_df['y_pred']!=4)]

### Erros Analysis for class 2 samples that are misclassified

In [ ]:
error_analysis_df.loc[error_analysis_df['y_train']==2, ['y_train', 'y_pred']].value_counts()

The model is confusing class 2 with class 1.

In [ ]:
error_analysis_df.loc[(error_analysis_df['y_train']==2) & (error_analysis_df['y_pred']!=2)]

These mistakes are very logical and supports our conclusion using ANOVA and t-SNE when we found that these multiclass classification problem can designed as binary classification by considering two classes low risk (scores 1 and 2) and high risk (scores 3, 4, and 5).

Generally in misclassified 4 and 2 class samples, the differences between the probability of the true class and the predicted class are very close. 

So, we will increase the probability of both classes 4 and 2 a little bit. 

### Create custom model with adjusted probabilities

In [ ]:
class AdjustedProbClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model, thresholds):
        self.model = model
        self.thresholds = thresholds

    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def predict(self, X):
        y_proba = self.predict_proba(X)
        preds = []
        for probs in y_proba:
            predicted_class = np.argmax(probs / np.array(self.thresholds)) + 1
            preds.append(predicted_class)
        return np.array(preds)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

### Increasing the probability of class 2 and 4 to increase their recall

In [ ]:
thresholds_grid = []
thresholds_to_try = [0.35, 0.37, 0.39, 0.41, 0.43]
for class2_threshold in thresholds_to_try:
    for class4_threshold in thresholds_to_try:
        thresholds_grid.append([0.5, class2_threshold, 0.5, class4_threshold, 0.5])

for thresholds in thresholds_grid:
    adjusted_prob_clf = AdjustedProbClassifier(xgb_clf, thresholds)
    y_pred = cross_val_predict(adjusted_prob_clf, X_train_preprocessed, y_train-1, 
                               cv=4, n_jobs=-1)
    print(f"Thresholds: {thresholds}")
    print(f"Classification Report:\n{classification_report(y_train, y_pred)}")
    print('-' * 70)

It seems that the best thresholds combination is: [0.5, 0.43, 0.5, 0.43, 0.5] (last one)

In [ ]:
pipeline_adjusted_prob = Pipeline([
    ('cleaning_pipeline', cleaning_pipeline),  
    ('feature_engineering', FeatureEng()), 
	('model', adjusted_prob_clf)
])

pipeline_adjusted_prob.fit(X_train_split, y_train_split-1)

In [ ]:
y_train_pred_adjusted_prob = pipeline_adjusted_prob.predict(X_train_split) 

In [ ]:
print('train_accuracy:', accuracy_score(y_train_split, y_train_pred_adjusted_prob), '\n')
print('train_classification_report:\n', classification_report(y_train_split, y_train_pred_adjusted_prob), '\n')

In [ ]:
y_val_pred_adjusted_prob = pipeline_adjusted_prob.predict(X_val_split) 

In [ ]:
print('test_accuracy:', accuracy_score(y_val_split, y_val_pred_adjusted_prob), '\n')
print('test_classification_report:\n', classification_report(y_val_split, y_val_pred_adjusted_prob), '\n')

This model with adjusted probabities seems to do slightly better in terms of f1_score and has more balanced precision and recall values. 

It may be better than the score of the 1st winner of the competition made on this dataset. [He built a model with **0.77** f1_macro](https://www.hackerearth.com/challenges/new/competitive/hackerearth-machine-learning-challenge-predict-customer-churn/#:~:text=Machine%20Learning%20practice-,Winners,-Adarsh%20Wase); however, my model has f1_macro of **0.78**

So, let's use this model for deployment. 

#### Is tuning probabilities useless? (in the context of business goals, not competitions)

I think that tuning probabilities **in this case** is not that useful because when the model missclassifies class 4 samples, it predicts them as class 3 or class 5 (all of them belong to the same category: high risk score) and when it misclassifies class 2 samples, it predicts them as class 1, which is also a low risk score. The point is that the model doesn't mix high scores with low scores. The model's misclassifications are "within-category".  

#### Group-level accuracy (validation of the argument in the previous cell: whether tuning probabilities useless) 

In [ ]:
# 'high risk score' is encoded as 1 and 'low risk score' is encoded as 0
y_train_binary = y_train.apply(lambda x: 1 if x >= 3 else 0)
y_train_binary.value_counts()

In [ ]:
xgb_clf_binary = XGBClassifier(verbosity=0, random_state=42, n_estimators=100, 
                       learning_rate=0.01, max_depth=6, subsample=0.75)

In [ ]:
cv_results_binaryClassification = cross_validate(xgb_clf_binary, X_train_preprocessed, y_train_binary, 
                                                cv=4, scoring='accuracy', return_train_score=True)
print(f"Train Accuracy (Binary Classification): {cv_results_binaryClassification['train_score'].mean():.4f} (+/- {cv_results_binaryClassification['train_score'].std():.4f})")
print(f"Validation Accuracy (Binary Classification): {cv_results_binaryClassification['test_score'].mean():.4f} (+/- {cv_results_binaryClassification['test_score'].std():.4f})")

In [ ]:
pipeline_binary = Pipeline([
    ('cleaning_pipeline', cleaning_pipeline),  
    ('feature_engineering', FeatureEng()), 
	('model', xgb_clf_binary)
])

pipeline_binary.fit(X_train_split, y_train_binary)

In [ ]:
y_val_pred_binary = pipeline_binary.predict(X_val_split)

In [ ]:
y_val_binary = y_val_split.apply(lambda x: 1 if x >= 3 else 0)
y_val_binary.value_counts()

In [ ]:
print('Test Accuracy (Binary Classification):', accuracy_score(y_val_binary, y_val_pred_binary), '\n')

The argument that we should not focus that much on tuning probabilities in this multiclassification problem seems to be true only if the business will make the same decisions for high-risk scores (3, 4, and 5) and same decisions for low-risk scores (1 and 2).

### Saving the final ML pipeline

In [ ]:
joblib.dump(pipeline, '/kaggle/working/full_xgb_pipeline.joblib')
joblib.dump(pipeline_adjusted_prob, '/kaggle/working/full_xgb_pipeline_adjusted_prob.joblib')